In [1]:
import yfinance as yf
from twelvedata import TDClient
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt

# 设置Twelve Data API密钥（从twelvedata.com获取，免费基本层）
TWELVE_DATA_API_KEY = '321a723f61624e6eb401655e5bce50fb'
td = TDClient(apikey=TWELVE_DATA_API_KEY)

In [2]:
# 美国：原油期货（近月和远月通过连续合约近似；在实践中，使用特定到期日）
us_ticker_near = 'CL=F'  # 原油连续
us_data = yf.download(us_ticker_near, start='2020-01-01', end='2025-08-02')
# 模拟价差：用于演示，使用移位收盘作为'远月'（实际：获取特定合约）
us_data['Near'] = us_data['Close']
us_data['Far'] = us_data['Close'].shift(-21)  # 约1个月移位（21天）
us_data['Spread'] = us_data['Near'] - us_data['Far']
us_data.dropna(inplace=True)
print(us_data.head())

C:\Users\86189\AppData\Local\Temp\ipykernel_37168\3459325473.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  us_data = yf.download(us_ticker_near, start='2020-01-01', end='2025-08-02')
[*********************100%***********************]  1 of 1 completed

Price           Close       High        Low       Open   Volume       Near  \
Ticker           CL=F       CL=F       CL=F       CL=F     CL=F              
Date                                                                         
2020-01-02  61.180000  61.599998  60.639999  61.599998   486873  61.180000   
2020-01-03  63.049999  64.089996  61.130001  61.180000   885861  63.049999   
2020-01-06  63.270000  64.720001  62.639999  63.709999   724236  63.270000   
2020-01-07  62.700001  63.150002  62.110001  62.910000   582649  62.700001   
2020-01-08  59.610001  65.650002  59.150002  62.840000  1205710  59.610001   

Price             Far     Spread  
Ticker                            
Date                              
2020-01-02  50.110001  11.070000  
2020-01-03  49.610001  13.439999  
2020-01-06  50.750000  12.520000  
2020-01-07  50.950001  11.750000  
2020-01-08  50.320000   9.290001  


In [5]:
# 中国：大连交易所铁矿石期货通过Twelve Data（使用连续合约符号"I:DCE"；如果报错，请检查API文档或替换为支持的符号如"A:DCE" for soybean）
china_ts = td.time_series(symbol="I:DCE", interval="1day", start_date="2020-01-01", end_date="2025-08-02").as_pandas()  # 连续铁矿石
# 模拟价差：使用移位'close'作为'远月'
china_data = pd.DataFrame()
china_data['Near'] = china_ts['close']
china_data['Far'] = china_ts['close'].shift(-21)  # 约1个月移位
china_data['Spread'] = china_data['Near'] - china_data['Far']
china_data.dropna(inplace=True)
print(china_data.head())

TwelveDataError: **symbol** or **figi** parameter is missing or invalid. Please provide a valid symbol according to API documentation: https://twelvedata.com/docs#reference-data

In [4]:
# 中国：大连交易所铁矿石期货通过Quandl
china_near = ndl.get('DCE/I202509')  # 示例：2025年9月合约；调整代码
china_far = ndl.get('DCE/I202512')   # 12月
china_data = pd.concat([china_near['Settle'], china_far['Settle']], axis=1)
china_data.columns = ['Near', 'Far']
china_data['Spread'] = china_data['Near'] - china_data['Far']
china_data.dropna(inplace=True)
print(china_data.head())

DataLinkError: (Status 403) Something went wrong. Please try again. If you continue to have problems, please contact us at connect@data.nasdaq.com.